Generate keywords based on queries

In [1]:
import pandas as pd

In [30]:
topics = pd.read_xml("data/topics-rnd5_covid-complete.xml")
topics

,number,query,question,narrative
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...
5,6,coronavirus test rapid testing,what types of rapid testing for Covid-19 have ...,Looking for studies identifying ways to diagno...
6,7,serological tests for coronavirus,are there serological tests that detect antibo...,Looking for assays that measure immune respons...
7,8,coronavirus under reporting,how has lack of testing availability led to un...,Looking for studies answering questions of imp...
8,9,coronavirus in Canada,how has COVID-19 affected Canada,"seeking data related to infections (confirm, s..."
9,10,coronavirus social distancing impact,has social distancing had an impact on slowing...,seeking specific information on studies that h...


In [3]:
words_total = []
for query in topics["query"].to_list():
    words = query.split(" ")
    words_total.extend(words)
    

In [48]:
from ElasticSettings.analyze_settings import settings
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
from credentials import username, password

In [49]:
print(settings)

{'analysis': {'analyzer': {'my-analyzer': {'type': 'custom', 'tokenizer': 'standard', 'filter': ['lowercase', 'stop', 'synonym', 'snowballer']}}, 'filter': {'snowballer': {'type': 'snowball', 'language': 'English'}, 'synonym': {'type': 'synonym', 'synonyms_path': 'synonyms.txt'}}}, 'similarity': {'my-similarity': {'type': 'BM25', 'k1': 1, 'b': 0.6}}}


In [50]:
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=(username, password)
)

In [51]:
index_name = "analyzer"
es.indices.delete(index=index_name)
es.indices.create(index=index_name, settings=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'analyzer'})

In [52]:
char_filter = {
    "type": "pattern_replace",
    "pattern": "-",
    "replacement": ""
}



In [72]:
key_words = set()

for query in topics["query"].to_list():
    # response = client.analyze(char_filter=char_filter, text=query)["tokens"][0]["token"]
    response_2 = es.indices.analyze(index=index_name, text=query, tokenizer="standard", filter=["lowercase", "stop", {"type": "synonym", "synonyms_path": "synonyms.txt"}, "unique", {"type": "stop", "ignore_case": True, "stopwords": ["coronavirus"]}])
    for token in response_2["tokens"]:
        key_words.add(token["token"])

In [77]:
key_words -= set(["19", "2"])

In [79]:
import json
json.dump(list(key_words), open("covid_topic_keywords.json", "w"))